# <center> Génération de données audio </center>

Nous avons besoin de générer des données audio contenant des erreurs grammaticales. Nous avons décidé d'utiliser une voix ayant un accent indien car il n'y avait pas d'accent français pour du texte anglais de disponible sur le TTS qu'on utilise. Cette caractéristique permettra par la suite d'affiner notre modèle STT à cet accent spécifique. Nous utilisons un jeu de données JFLEG qui contient des phrases erronées en anglais et leurs corrections. Nous allons utiliser un TTS pour obtenir ces phrases erronées à l'oral et ainsi obtenir notre jeu de données pour notre pipeline.

In [2]:
# !pip install gTTS datasets

     -------------------------------------- 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     --------------------------------------- 24.6/24.6 MB 20.4 MB/s eta 0:00:00
     ------------------------------------- 365.5/365.5 kB 22.2 MB/s eta 0:00:00
     ---------------------------------------- 76.2/76.2 kB ? eta 0:00:00
     ---------------------------------------- 44.7/44.7 kB 2.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, concatenate_datasets, Dataset
from datasets.features.audio import Audio
import pandas
from gtts import gTTS

In [4]:
dataset = load_dataset("jfleg")
dataset

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})

Le dataset de JFLEG est composé d'un jeu de validation et d'un jeu de test. Nous concaténons ces deux ensembles pour les rediviser en jeu d'entrainement et de test avec les proportions que nous choisirons nous même plus tard.

In [5]:
dataset_concat = concatenate_datasets([dataset["validation"], dataset["test"]])
dataset_concat

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 1503
})

On va faire un prétraitement des données car certains textes de jfleg sont vides (indice 754 par exemple) et bloquent donc la création des audios.

In [6]:
dataset_concat["sentence"][754]

''

In [7]:
# Filtrer les textes vides dans dataset_concat
dataset_concat = dataset_concat.filter(lambda example: example["sentence"].strip() != "")

Filter:   0%|          | 0/1503 [00:00<?, ? examples/s]

In [9]:
dataset_concat["sentence"][754]

'New and new technology has been introduced to the society .'

Nous retranscrivons maintenant chaque phrase en audio grâce au modèle TTS de google :

In [10]:
import os
audio_folder = "./audio"

# si les audios n'existent pas déjà, les créer
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)
    for i, sentence in enumerate(dataset_concat['sentence']):
      if sentence != "" or sentence is not None:
        print(sentence)
        tts = gTTS(sentence, lang='en', tld='co.in')
        tts.save(audio_folder+'/{}.mp3'.format(i))
else:
   print("Les retranscriptions audios se trouvent dans le dossier 'audio'. ")

So I think we can not live if old people could not find siences and tecnologies and they did not developped . 
For not use car . 
Here was no promise of morning except that we looked up through the trees we saw how low the forest had swung . 
Thus even today sex is considered as the least important topic in many parts of India . 
image you salf you are wark in factory just to do one thing like pot taire on car if they fire you you will destroy , becouse u dont know more than pot taire in car . 
They draw the consumers , like me , to purchase this great product with all these amazing ingredients and all that but actually they just sometimes make something up just to increase their sales . 
I want to talk about nocive or bad products like alcohol , hair spray and cigarrets . 
For example they can play football whenever they want but the olders can not . 
It figures Diana Krall wearing a Rolex watch and has a text that suggests that if the reader wants to belong to the restricted club , n

gTTSError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown

In [11]:
nb_audio=0
x=[]
for file in os.listdir(audio_folder):
    if file.endswith('.mp3'):
        x.append(file)
        nb_audio+=1
print("Nombre total d'audio créé: {} sur {} textes du jeu de données JFLEG".format(str(nb_audio), len(dataset_concat["sentence"])))

Nombre total d'audio créé: 1142 sur 1501 textes du jeu de données JFLEG


L'API de notre STT nous empêche de convertir l'ensemble de nos données, mais 1142 audios sur les 1501 textes initiaux représente déjà une belle proportion d'audios.